In [12]:
import pandas as pd

In [13]:
# Import csv file
df = pd.read_csv('2016-2023-imi-results-long-view.csv')

# Remove missing values (NaN or None) from a DataFrame
new_df = df.dropna()

df_filtered_cols = new_df.drop(['HOSPITAL', 'OSHPDID', 'Procedure/Condition',
                               'Risk Adjuested Mortality Rate',
                               'Hospital Ratings', 'LONGITUDE', 'LATITUDE'], axis=1)

# Change # of Deaths and Cases columns from strings to ints
df_filtered_cols["# of Deaths"] = pd.to_numeric(df_filtered_cols["# of Deaths"])
df_filtered_cols["# of Cases"] = pd.to_numeric(df_filtered_cols["# of Cases"])

# Group columns by Year and County and sum the # of Deaths values of each row grouped
df1 = df_filtered_cols.groupby(["YEAR","COUNTY"])["# of Deaths"].sum()

# Group columns by Year and County and sum the # of Cases of each row grouped
df2 = df_filtered_cols.groupby(["YEAR","COUNTY"])["# of Cases"].sum()

# Combine columns of both # of Cases and Deaths
df_cleaned = pd.concat([df1, df2], axis=1)

print(df_cleaned)

# Export to a csv file
df_cleaned.to_csv("cleaned_2016-2023-imi-results-long-view.csv")

                # of Deaths  # of Cases
YEAR COUNTY                            
2016 Alameda           1622       34012
     Amador              38         790
     Butte              394       10032
     Calaveras           10         310
     Colusa               2          14
...                     ...         ...
2023 Tulare             248        5089
     Tuolumne            23         671
     Ventura            413        9575
     Yolo                26         912
     Yuba                95        2265

[438 rows x 2 columns]


In [14]:
# Select columns to read from csv file
cols_to_use = [0,2,3,4,6,8]

# Import csv file
df = pd.read_csv('20230413_adult_demo_race_supp.csv', usecols=cols_to_use)

# Remove missing values (NaN or None) from a DataFrame
new_df = df.dropna()

print(new_df)

# Export to a csv file
new_df.to_csv("cleaned_20230413_adult_demo_race_supp.csv", index=False)

                POPULATION_NAME  FISCAL_YEAR  \
0                       Alameda         2019   
1                       Alameda         2019   
2                       Alameda         2019   
3                       Alameda         2019   
4                       Alameda         2019   
...                         ...          ...   
18361  Very Large Counties (LA)         2022   
18362  Very Large Counties (LA)         2022   
18363  Very Large Counties (LA)         2022   
18364  Very Large Counties (LA)         2022   
18365  Very Large Counties (LA)         2022   

                                   RACE_GRP   TOTAL_CT  MHS1_CT  MHS5_CT  
0      R1 Alaskan Native or American Indian      851.0    109.0     25.0  
1              R2 Asian or Pacific Islander    81208.0   4318.0    911.0  
2                                  R3 Black    53823.0   6484.0   1528.0  
3                               R4 Hispanic    73645.0   4357.0    891.0  
4                                  R5 White    4

In [15]:
# Import csv file
df = pd.read_csv('4.7-cohs-plans-q4-2024-odp.csv')

# Create a dictionary d
d = {}

# Add the Hospital key to County value pairs
d["CalOPTIMA/Orange"] = "Orange"
d["CalOptima/Orange"] = "Orange"
d["Central California Alliance for Health"] = "San Benito, Santa Cruz, Monterey, Merced, Mariposa"
d["Gold Coast Health Plan"] = "Ventura"
d["Partnership Health Plan of CA"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Partnership Health Plan of CA/"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Partnership Health Plan of California/"] = "Del Norte, Colusa, Butte, Placer, Plumas, Siskiyou, Lassen, Solano, Lake, Sierra, Shasta, Glenn, Humboldt, Napa, Yolo, Nevada, Yuba, Trinity, Modoc, Sutter, Mendocino, Tehama, Sonoma, Marin"
d["Santa Barbara Health Authority"] = "Santa Barbara"
d["CenCal - San Luis Obispo"] = "San Luis Obispo"
d["CenCal - Santa Barbara"] = "Santa Barbara"
d["CenCal Health/San Luis Obispo"] = "San Luis Obispo"
d["CenCal Health/Santa Barbara"] = "Santa Barbara"
d["Gold Coast Health Plan/Ventura"] = "Ventura"
d["Health Plan of San Mateo"] = "San Mateo"
d["Health Plan of San Mateo/San Mateo"] = "San Mateo"
d["Partnership Health Plan/Yuba"] = "Yuba"

# Replace values in DataFrame using dictionary d
for key, value in d.items():
  df["County Organized Health System (COHS) Health Plan"] = df["County Organized Health System (COHS) Health Plan"].apply(lambda x: value if x == key else x)

# Seperate Kaiser Permanente/{County} row values into seperate rows
print(df)
for company in df["County Organized Health System (COHS) Health Plan"]:
  if "Kaiser Permanente/" in company:
    county = company.split("/")[1]
  else:
    county = company
  df["County Organized Health System (COHS) Health Plan"] = df["County Organized Health System (COHS) Health Plan"].apply(lambda x: county if x == company else x)
print(df)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
  year = row["Year"]
  period = row["Reporting Period"]
  county = row["County Organized Health System (COHS) Health Plan"]
  eligible = row[" Number of Eligible Individuals "]

  # Split rows with multiple counties into single rows and drop original row
  counties = county.split(", ")
  if len(counties) > 1:
    i = df[((df["Year"] == year)
    & (df["Reporting Period"] == period)
    & (df["County Organized Health System (COHS) Health Plan"] == county)
    & (df[" Number of Eligible Individuals "] == eligible))].index
    df.drop(i, inplace=True)

    # Add the new row with the singular county
    for countyIndividual in counties:
      df.loc[len(df)] = [year, period, countyIndividual, eligible]


# Drop the Reporting Period column
df_filtered_cols = df.drop(['Reporting Period'], axis=1)

# Convert Number of Eligible Individuals numbers with commas to numbers without commas
df_filtered_cols[' Number of Eligible Individuals '] = df_filtered_cols[' Number of Eligible Individuals '].str.findall(r'[^\s,](?:[^,]*[^\s,])?').str.join('')

# Change Number of Eligible Individuals column from strings to ints
df_filtered_cols[" Number of Eligible Individuals "] = pd.to_numeric(df_filtered_cols[" Number of Eligible Individuals "])

# Group columns by Year and County Organized Health System (COHS) Health Plan and sum the Number of Eligible Individuals values of each row grouped
df_filtered_cols = df_filtered_cols.groupby(["Year","County Organized Health System (COHS) Health Plan"])[" Number of Eligible Individuals "].sum()

# Remove missing values (NaN or None) from a DataFrame
new_df = df_filtered_cols.dropna()

print(new_df)

# Export to a csv file
new_df.to_csv("cleaned_4.7-cohs-plans-q4-2024-odp.csv")

     Year Reporting Period  County Organized Health System (COHS) Health Plan  \
0    2016          2016 Q1                                             Orange   
1    2016          2016 Q1  San Benito, Santa Cruz, Monterey, Merced, Mari...   
2    2016          2016 Q1                                            Ventura   
3    2016          2016 Q1                                          San Mateo   
4    2016          2016 Q1  Del Norte, Colusa, Butte, Placer, Plumas, Sisk...   
..    ...              ...                                                ...   
274  2024          2024 Q4                           Kaiser Permanente/Solano   
275  2024          2024 Q4                           Kaiser Permanente/Sonoma   
276  2024          2024 Q4                          Kaiser Permanente/Ventura   
277  2024          2024 Q4                             Kaiser Permanente/Yolo   
278  2024          2024 Q4  Del Norte, Colusa, Butte, Placer, Plumas, Sisk...   

     Number of Eligible Ind

In [16]:
# Import csv file
df = pd.read_csv('data-infant-mortality.csv')

# Drop the Category, Subcategory, Rate, Indicator Name,
# Lower 95% Confidence Limit, Upper 95% Confidence Limit,
# Unnamed: 10, Unnamed: 11, Unnamed: 12, Unnamed: 13, Unnamed: 14 columns
df_filtered_cols = df.drop(['Category', 'Subcategory',
                            'Rate', 'Indicator Name',
                               'Lower 95% Confidence Limit',
                               'Upper 95% Confidence Limit',
                                'Unnamed: 10', 'Unnamed: 11',
                                'Unnamed: 12', 'Unnamed: 13',
                                'Unnamed: 14'], axis=1)

# Drop the first 2 rows and reset the index values
df_filtered_cols = df_filtered_cols.iloc[2:].reset_index()

# Drop the index column
df_filtered_cols = df_filtered_cols.drop(['index'], axis=1)

# Group columns by Geography and Year and sum the Numerator values of each row grouped
df1 = df_filtered_cols.groupby(["Geography","Year"])["Numerator"].sum()

# Group columns by Geography and Year and sum the Denominator values of each row grouped
df2 = df_filtered_cols.groupby(["Geography","Year"])["Denominator"].sum()

# Combine columns of both Numerator and Denominator
df_cleaned = pd.concat([df1, df2], axis=1)

# Remove missing values (NaN or None) from a DataFrame
new_df = df_cleaned.dropna()

print(new_df)

# Export to a csv file
new_df.to_csv("cleaned_data-infant-mortality.csv")

                     Numerator  Denominator
Geography Year                             
Alameda   2007           107.0      21522.0
          2007-2009     4913.0    1378297.0
          2008            81.0      20977.0
          2008-2010     4804.0    1331159.0
          2009            82.0      20326.0
...                        ...          ...
Yuba      2018-2020      281.0      80819.0
          2019            12.0       1167.0
          2019-2021      278.0      82619.0
          2020             0.0       1116.0
          2021             0.0       1181.0

[1678 rows x 2 columns]
